In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:


Number_level = 10;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];


In [3]:
function reward(s, a,k,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
    p=[sum(1-T[i,i,1,j] for i in 1:9) for j in 1:NumberUnits]/sum(sum(1-T[i,i,1,j] for i in 1:9) for j in 1:NumberUnits)   
    if s==Number_level
            r=failure_penalty +  setup_cost/NumberUnits  + normal_operation  ;
        return r
       end
        r=0
     if a==1
        if s1>=s
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
         return r
        else
            return 0.0
        end
        end
    if a==2
         if s1>=s
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits ;
         return r
        else
            return 0.0
        end
        end
    if a==3
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits 
         crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
        return r
    end
    end

reward (generic function with 3 methods)

In [4]:
T=zeros(Number_level,Number_level,3,100);

In [5]:
for i in 1:100
    fullname = "TM_MATRIX/tm"*string(i);
    # generate_transition_matrix(Number_level, fullname) 
    Transition_matrix = load(fullname*".jld2","transition_matrix");
    
    T[:,:,1,i].=Transition_matrix;
    T[:,:,2,i].=Transition_matrix;
    for j in 1:10
        T[j,:,3,i].=Transition_matrix[1,:];
        end
    T[10,:,1,i].=Transition_matrix[1,:]
T[10,:,2,i].=Transition_matrix[1,:]
end

In [6]:

function generative(s, a,k, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
        if s==Number_level
        s=1
        crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
        return (sp=s, r=r)
       end
        r=0
     if a==1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
         return (sp=s, r=r)
        end
    if a==2
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits ;
             return (sp=s, r=r)
        end
    if a==3

        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r+= normal_operation  ;
       return (sp=s, r=r)
    end
    end

generative (generic function with 2 methods)

In [7]:
function ValueIteration(γ)
    Q=zeros((Number_level,3,NumberUnits))
#     print(Q)
    V= rand(Float64, (Number_level,NumberUnits))*1000
    diff=0
    i=0
    
    

    
    
    for k in 1:NumberUnits
        p=[1 1 1 ]
    for i in 1:1500
    for s in 1:Number_level
        Threads.@threads for a in 1:3
                Q[s,a,k]= (reward(s,a,k)) + γ*( sum(T[s,s1,a,k]*(V[s1,k]) for (s1) in 1:Number_level ))
        end
            p=[exp(lamb*sum(Q[s,1,k])),exp(lamb*(Q[s,2,k])),exp(lamb*(Q[s,3,k]))]
            p=p./sum(p)
       V[s,k]=findmax(Q[s,1:3,k])[1]

        end
        
        end
    end
    return Q
    end

ValueIteration (generic function with 1 method)

In [8]:



function decison(Q, s,n=0.2)
   a_final=[]
   if Number_level in s
        for i in 1:NumberUnits
            append!(a_final,argmax(a->Q[s[i],a,i], [2,3]))
            end
        return a_final
    end
    q0=0.0
    a_0=repeat(1:1,NumberUnits);
    for i in 1:NumberUnits
            q0+=Q[s[i],1,i]
    end
    a_final=repeat(1:1,NumberUnits);
    b=1
    for i in 1:NumberUnits
            a_final[i]=argmax(a->Q[s[i],a,i], [2,3])

            if a_final[i]==3
                b+=1
            end
    end
     
        q1=0.0
    for i in 1:NumberUnits
            q1+=Q[s[i],a_final[i],i]
    end
    if !(3 in a_final) 
        a_final=a_0
    end
    if q0>q1
            return a_0
        else
            return a_final
        end
end

decison (generic function with 2 methods)

In [9]:
using DataFrames
df= DataFrame(Units=[],n=[],s=[],m=[],f=[],lambda=[],mean=[],std=[])

0×8 DataFrame

In [10]:


# print(Q)
cost=[ 
    
    [0 -800 -200 -1000]
    ]

for units in [30,40]
     for c in cost
   print(units)  
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty])   
     for lambda in 1
        global  lamb=lambda
    global NumberUnits=units
     Q=ValueIteration(.95)
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
 Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
            r2=0.0
    for i in 1:NumberUnits
         s[i],r[i]=generative(s[i],a[i],i,234);
          r2+=r[i]
            end 
    s=s;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
    results[h]=rk
end
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,normal_operation,setup_cost,maintenance_penalty,failure_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end

30[0, -800, -200, -1000]


┌ Info:  started timer at: 2023-04-17T11:27:59.404
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


30
1
-57488.95674916878
3001.7002023654836
-------------------------
40[0, -800, -200, -1000]


┌ Info:           46.1415803s: 46 seconds, 141 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62
┌ Info:  started timer at: 2023-04-17T11:38:38.671
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


40
1
-70752.97424441637
3262.909098773062
-------------------------


┌ Info:           58.3381118s: 58 seconds, 338 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62


In [11]:
df

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any
1,40.0,0.0,-800.0,-200.0,-1000.0,1.0,-70753.0,3262.91
2,30.0,0.0,-800.0,-200.0,-1000.0,1.0,-57489.0,3001.7


In [12]:
using CSV
CSV.write("CW-MDP_HETERO.csv", df)

"CW-MDP_HETERO.csv"